# Spectrogram-visualizer from embedded data

In [1]:

import serial
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import IPython
import pipeline.config as conf
import octafilt3r.filter as o3f

param_dict = conf.open_params()
srl = serial.Serial("COM11", 115200)


In [2]:
fcs, fls, fus, n_bands = o3f._gen_fc_fl_fu(param_dict["fmax"], 
                                           param_dict["fmin"], 
                                           param_dict["oct_bw_ratio"])

init_data = np.random.rand(param_dict["n_bands"], param_dict["fps"])
ylabels = []
bin2freq = np.arange(param_dict["n_bands"])
for i in range(len(fcs)):
    ylabels.append(str(int(fcs[i])))

In [3]:

fig, ax = plt.subplots(figsize=(12, 8))
plt.title("Spectroviewer")
plt.xlabel("Frames")
plt.ylabel("Hz")
ax.set_yticks(bin2freq)
ax.set_yticklabels(ylabels)
im = ax.imshow( init_data, interpolation='nearest', 
                origin='lower', 
                aspect='auto', # get rid of this to have equal aspect
                vmin=np.min(param_dict["threshold"]),
                vmax=np.max(5), 
                cmap='rainbow')
cb = plt.colorbar(im, label='dBFS-LEQ')

srl.flushInput()

def animate(i):
    spec = []
    if(srl.inWaiting() != 0):
        raw = str(srl.readline())[2:-5]
        srl.flushInput()
        chunks = raw.split(' ')[:-1]
        if(np.asarray(chunks).shape != (700,)):
            return None

        for i in range(25):
            frame = [float(numeric_string) for numeric_string in chunks[28 * i: 28 * (i + 1)]]
            spec.append(np.asarray(frame[::-1]))
            
        spec = np.asarray(spec).T  
        im.set_data(spec)

ani = animation.FuncAnimation(fig, animate, interval=1000)
plt.show()
